In [129]:
import torch
import numpy as np
import pandas as pd
import sys
import string
from collections import Counter
import nltk
import math
import torch
import json
import torch.nn.functional as F
from tqdm.auto import tqdm
import itertools as it
import random
import json, requests

sys.path.append('../range_matching')
# import faq_service as fs
import knrm
from importlib import reload

In [127]:
reload(knrm)
glue_qqp_dir = '/home/nur/projects/analysis/range_matching/data/QQP'
glove_path = '/home/nur/projects/analysis/range_matching/data/glove.6B.50d.txt'

def get_glue_df(glue_qqp_dir, partition_type: str) -> pd.DataFrame:
    assert partition_type in ['dev', 'train']
    glue_df = pd.read_csv(
        glue_qqp_dir + f'/{partition_type}.tsv', sep='\t', error_bad_lines=False, dtype=object)
    glue_df = glue_df.dropna(axis=0, how='any').reset_index(drop=True)
    glue_df_fin = pd.DataFrame({
        'id_left': glue_df['qid1'].astype('int32'),
        'id_right': glue_df['qid2'].astype('int32'),
        'text_left': glue_df['question1'],
        'text_right': glue_df['question2'],
        'label': glue_df['is_duplicate'].astype('int32')
    })
    return glue_df_fin

df = get_glue_df(glue_qqp_dir, 'dev')
df.head(1)

,id_left,id_right,text_left,text_right,label
0,303345,303346,Why are African-Americans so beautiful?,Why are hispanics so beautiful?,0


In [154]:
def gen_unique_text(dflist):
    idset = set()
    for df in dflist:
        dfcut = df[~df.id_left.isin(idset) | ~df.id_right.isin(idset)]
        for _, row in tqdm(dfcut.iterrows(), total=dfcut.shape[0]):
            if row.id_left not in idset:
                yield str(row.id_left), row.text_left
                idset.add(row.id_left)
            if row.id_right not in idset:
                yield str(row.id_right), row.text_right
                idset.add(row.id_right)

gen = gen_unique_text([df])
text_list = list(gen)
len(text_list)

  0%|          | 0/40430 [00:00<?, ?it/s]

73349

In [235]:
num_query = 100
num_index = len(text_list) - num_query
random.shuffle(text_list)

payload_index = {"documents": {idx: text for idx, text in text_list[:num_index]}}

queries_idxs = [int(idx) for idx,_ in text_list[-num_query:]]
payload_query = {"queries": [text for _, text in text_list[-num_query:]]}
len(payload_index['documents']), len(payload_query['queries'])

(73249, 100)

In [250]:
payload = req
r = requests.post('http://localhost:11000/update_index', json=json.dumps(payload_index))
r, r.content

(<Response [200]>, b'{"status": "ok", "index_size": 73249}')

In [251]:
payload = req
r = requests.post('http://localhost:11000/query', json=json.dumps(payload_query))
r

<Response [200]>

In [252]:
query_idx_hat = []
for idx in queries_idxs:
    a = df[df.id_left == idx].id_right.values.tolist()
    b = df[df.id_right == idx].id_left.values.tolist()
    query_idx_hat.append(a+b)


def eval(r, query_idx_hat):
    res = json.loads(r.content)['suggestions']
    hit_count = 0
    for suggest, y_hat in zip(res, query_idx_hat):
        if not suggest:
            continue
        idx_set = {int(idx) for idx, _ in suggest} 
        if idx_set & set(y_hat):
            hit_count +=1
    return hit_count / len(res)

eval(r, query_idx_hat)

0.49

In [200]:
for query, res, y_hat in zip(payload_query['queries'], json.loads(r.content)['suggestions'], query_idx_hat):
    print(query)
    for row in res:
        print(row)
    print("Expected", y_hat)
    print(df[df.id_left.isin(y_hat)].text_left.values)
    print(df[df.id_right.isin(y_hat)].text_right.values)
    print("-"*20)

What are some of the best videos you have ever seen?
['31540', 'What are the most amazing photos you have ever taken?']
['50766', 'What are some of the best movies of all times I should watch (animated are welcome)?']
['89899', 'What are some best movies of all time?']
['96077', 'What are some of the movies of Hollywood that you must watch?']
['425694', 'What are some of the best background pictures ever taken?']
['83946', 'Which are the best TV series that you have ever watched and why?']
['87754', 'Why are so many people asking about what is the most epic picture ever taken?']
['19508', 'What is the best movie you have seen that most people will probably never have heard of?']
['8513', 'What are some of the best jokes?']
['233777', 'What are some of the best start up business ideas that you have ever heard?']
Expected [123142]
['Which is the best video you have ever seen?']
[]
--------------------
What if a university professor loses his students' exams before publishing grades?
['28

TypeError: 'NoneType' object is not iterable